自定义状态

In [5]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.tools import tool
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langgraph.types import Command, interrupt

from llm_layer import llm

'''自定义状态，添加name和birthday属性'''
class State(TypedDict):
    messages: Annotated[list, add_messages]
    name: str
    birthday: str

graph_builder = StateGraph(State)

'''更新工具内部状态'''
@tool
# 请注意，由于我们正在生成用于状态更新的ToolMessage，因此我们通常需要相应工具调用的ID。
# 我们可以使用LangChain的InjectedToolCallId来表示此参数不应在工具的模式中向模型公开。
def human_assistance(
    name: str, birthday: str, tool_call_id: Annotated[str, InjectedToolCallId]
) -> str:
    """Request assistance from a human."""
    human_response = interrupt(
        {
            "question": "Is this correct?",
            "name": name,
            "birthday": birthday,
        },
    )
    # If the information is correct, update the state as-is.
    if human_response.get("correct", "").lower().startswith("y"):
        verified_name = name
        verified_birthday = birthday
        response = "Correct"
    # Otherwise, receive information from the human reviewer.
    else:
        verified_name = human_response.get("name", name)
        verified_birthday = human_response.get("birthday", birthday)
        response = f"Made a correction: {human_response}"

    # This time we explicitly update the state with a ToolMessage inside
    # the tool.
    state_update = {
        "name": verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response, tool_call_id=tool_call_id)],
    }
    # We return a Command object in the tool to update our state.
    return Command(update=state_update)

tool = TavilySearch(max_results=2)
tools = [tool, human_assistance]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    # Because we will be interrupting during tool execution,
    # we disable parallel tool calling to avoid repeating any
    # tool invocations when we resume.
    assert len(message.tool_calls) <= 1
    return {"messages": [message]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")


'''编译graph'''
memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

In [6]:
'''prompt the chatbot'''
user_input = (
    "Can you look up when LangGraph was released? "
    "When you have the answer, use the human_assistance tool for review."
)
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Can you look up when LangGraph was released? When you have the answer, use the human_assistance tool for review.
================================== Ai Message ==================================

[{'text': "I'll help you search for information about LangGraph's release and then get it reviewed.\n\nFirst, let me search for LangGraph's release date:", 'type': 'text'}, {'id': 'toolu_01NfXxyUKesZN8BgQLqU7RG2', 'input': {'query': 'When was LangGraph released launch date'}, 'name': 'tavily_search', 'type': 'tool_use'}]
Tool Calls:
  tavily_search (toolu_01NfXxyUKesZN8BgQLqU7RG2)
 Call ID: toolu_01NfXxyUKesZN8BgQLqU7RG2
  Args:
    query: When was LangGraph released launch date
================================= Tool Message =================================
Name: tavily_search

{"query": "When was LangGraph released launch date", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Cu

In [7]:
'''聊天机器人无法识别正确的日期'''
# 提供下面的信息
human_command = Command(
    resume={
        "name": "LangGraph",
        "birthday": "Jan 17, 2024",
    },
)

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================

[{'text': 'Based on my research, LangGraph appears to have been released recently, with major announcements and updates happening in early 2024. The LangGraph Studio IDE was recently announced, and there have been ongoing releases and updates. However, to ensure we have the most accurate information about the initial release date, let me request human assistance for review:', 'type': 'text'}, {'id': 'toolu_012TLmCYnBFc9eCWmM6MYwmJ', 'input': {'name': 'LangGraph', 'birthday': '2024-01-01'}, 'name': 'human_assistance', 'type': 'tool_use'}]
Tool Calls:
  human_assistance (toolu_012TLmCYnBFc9eCWmM6MYwmJ)
 Call ID: toolu_012TLmCYnBFc9eCWmM6MYwmJ
  Args:
    name: LangGraph
    birthday: 2024-01-01
================================= Tool Message =================================
Name: human_assistance

Made a correction: {'name': 'LangGraph', 'birthday': 'Jan 17, 2024'}
================================== Ai Mess

In [8]:
'''获取状态'''
snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{'name': 'LangGraph', 'birthday': 'Jan 17, 2024'}

In [9]:
'''更新状态'''
graph.update_state(config, {"name": "LangGraph (library)"})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f05327a-6163-68ee-8008-6f7ce362b05f'}}

In [10]:
'''查看更新的值'''
snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{'name': 'LangGraph (library)', 'birthday': 'Jan 17, 2024'}